In [9]:
#This is the notebook for tools_project

In [10]:
# scraping libraries to read html code
import requests
from bs4 import BeautifulSoup
import json

In [11]:
# pandas library to create dataframe
import pandas as pd
import numpy as np

In [58]:
# Direct to initial results
url = 'https://www.yelp.com/search?find_desc='
keyword = input('What do you want to eat now ? ')
address = input('Where are you ? ')

url = 'https://www.yelp.com/search?find_desc='+ keyword + '&find_loc=' + address 
url

What do you want to eat now ? burger
Where are you ? columbia university


'https://www.yelp.com/search?find_desc=burger&find_loc=columbia university'

In [59]:
# Checking if the url is "valid", the status code should be 200
response = requests.get(url)
response.status_code

200

In [60]:
# extract the html code from the web page = https://www.yelp.com/search?find_desc=Restaurants&find_loc=New+York,+NY&start=30
results_page = BeautifulSoup(response.content,'lxml')

In [61]:
# creating a list of businesses with the html code of the business as elements of the list
businesses = results_page.find_all('a',class_ = "biz-name js-analytics-click")

In [62]:
businesses

[<a class="biz-name js-analytics-click" data-analytics-label="biz-name" data-hovercard-id="w5qH0SZVZyioFvkpmA_tdQ" href="/adredir?ad_business_id=HIyfZAMlvjpVqrK4vbxIFA&amp;campaign_id=Aqe9fxTYJJQ0EDKiWFwm6g&amp;click_origin=search_results&amp;placement=above_search&amp;redirect_url=https%3A%2F%2Fwww.yelp.com%2Fbiz%2Fellis-cafe-new-york&amp;request_id=688181d7fef2abb2&amp;signature=3bcdf0a3d09f0cf93556d7e8d5f1188b01aefcf23fcd7980695b6be52f1345d9&amp;slot=0"><span>Ellis Cafe</span></a>,
 <a class="biz-name js-analytics-click" data-analytics-label="biz-name" data-hovercard-id="mdgE9acBvCypEKhJNe0L4g" href="/biz/mels-burger-bar-new-york?osq=burger"><span>Mel’s Burger Bar</span></a>,
 <a class="biz-name js-analytics-click" data-analytics-label="biz-name" data-hovercard-id="1nTnVCf7dcqkukPeFJkC7A" href="/biz/shake-shack-new-york-54?osq=burger"><span>Shake Shack</span></a>,
 <a class="biz-name js-analytics-click" data-analytics-label="biz-name" data-hovercard-id="dUVhojSyRI3Y5NK7MWU5_w" href=

In [63]:
# selecting the first element (a particular restaurant in our case) of the list businesses and looking in the
# html code for its name
business = businesses[0]
name = business.find('span').get_text()

# getting the link to the yelp page this perticular restaurant
link = 'https://www.yelp.com' + business.get('href')

In [65]:
print(name,link)

Ellis Cafe https://www.yelp.com/adredir?ad_business_id=HIyfZAMlvjpVqrK4vbxIFA&campaign_id=Aqe9fxTYJJQ0EDKiWFwm6g&click_origin=search_results&placement=above_search&redirect_url=https%3A%2F%2Fwww.yelp.com%2Fbiz%2Fellis-cafe-new-york&request_id=688181d7fef2abb2&signature=3bcdf0a3d09f0cf93556d7e8d5f1188b01aefcf23fcd7980695b6be52f1345d9&slot=0


In [19]:
# extracting the html code of the web page associated with the first business of our list businesses
response_business = requests.get(business_link)
results_business_page = BeautifulSoup(response_business.content,'lxml')

In [20]:
# looking in the html page of the restaurant for latitude and longitude coordinates of the restaurant
# the json object json_map contains these coordinates
json_map = results_business_page.find('div', class_ = "lightbox-map hidden").get('data-map-state')

# we can convert this json object to a python dictionnary using the following method
dict_map = json.loads(json_map)

# the relevant keys in this dict are "markers", "location" and "latitude" and "longitude"
latitude = dict_map["markers"][1]["location"]["latitude"]
longitude = dict_map["markers"][1]["location"]["longitude"]

In [21]:
json_map

'{"serviceAreas": [], "moMapPossible": true, "scrollwheelZoom": false, "zoomControlPosition": "top_right", "minZoomlevel": null, "isFullBleed": false, "maxZoomlevel": null, "zoom": 15, "library": "google", "fitToGeobox": false, "hoods": [], "adPinColor": null, "markers": [{"location": null, "key": "directions_marker", "icon": {"name": "directions", "anchorOffset": [12, 32], "activeOrigin": [0, 0], "scaledSize": [24, 32], "regularUri": "https://s3-media4.fl.yelpcdn.com/assets/srv0/yelp_maps/79f63ebc20db/assets/img/directions@2x.png", "size": [24, 32], "activeUri": "https://s3-media4.fl.yelpcdn.com/assets/srv0/yelp_maps/7249ab345ac8/assets/img/directions_highlighted@2x.png", "regularOrigin": [0, 0]}}, {"resourceType": "business", "url": "/biz/cafe-mogador-brooklyn", "resourceId": "jAaVnUKLITkuhzwXIe0vLQ", "shouldOpenInNewTab": false, "location": {"latitude": 40.719747, "longitude": -73.959993}, "key": "starred_business", "hovercardId": "2Fg2iUXmKBdN5TF8bE0rlg", "icon": {"name": "starred"

In [26]:
print(latitude,longitude)

40.7354179 -73.9829477


In [31]:
# The category can be found in the 'a' tag under "span", class_ = 'category-str-list' tag, 
category= results_business_page.find("span", class_ = 'category-str-list').find('a').get_text()

In [32]:
category

'Moroccan'

In [33]:
# the price range ('$','$$','$$$', or '$$$$') can be found in the "span", class_ = 'business-attribute price-range' tag
# the price range is not always available, that is why we need to catch errors 
# if necessary (hence the try: except) structure

try:
    price_range = results_business_page.find("span", class_ = 'business-attribute price-range').get_text()
except:
    price_range = None

In [34]:
print(price_range)

$$


In [35]:
# the ratings associated to a particular restaurant are in the "div", itemprop = 'aggregateRating' tag
# the rating value (between 0 and 5) can be accessed through ratings (= results_business_page.find("div", 
# itemprop = 'aggregateRating') at the tag "meta" and the attribute 'content'
# the number of reviews is available through ratings at the 'span' tag

ratings = results_business_page.find("div", itemprop = 'aggregateRating')
rating_value = ratings.find("meta").get('content')
review_count = ratings.find("span").get_text()

In [36]:
print(rating_value,review_count)

4.5 1004


In [37]:
# the phone number can be accessed at the tag 'span',itemprop = "telephone"
# phone numbers are not always available so we need to catch errors using the try:... except: structure

try:
    phone_number = results_business_page.find('span',itemprop = "telephone").get_text().strip()
except:
    phone_number = None   

In [38]:
print(phone_number)

(718) 486-9222


In [39]:
# We want to know if the restaurant is currently open: this information is 
# given at the 'span', class_ = "nowrap extra open" tag. This information is not always 
# available that is why we use the try: ... except:... structure

try:
    open_ = results_business_page.find('span', class_ = "nowrap extra open").get_text()
except:
    open_ = None

In [40]:
print(open_)

Open now


In [45]:
try:
    addr = results_business_page.find('strong', class_ = "street-address").get_text().strip()
except:
    addr = None
addr

'133 Wythe AveBrooklyn, NY 11211'

In [51]:
try:
    web = results_business_page.find('span', class_ = "biz-website js-biz-website js-add-url-tagging").find('a').get_text()
except:
    web = None
web

'cafemogador.com/williamsburg'

In [53]:
#want to put all infomations we want to get from one business in to one funtion
#input: url link
#output: all details we tested above
def get_info(link):
    response_business = requests.get(business_link)
    results_business_page = BeautifulSoup(response_business.content,'lxml')
    
    try:
        addr = results_business_page.find('strong', class_ = "street-address").get_text().strip()
    except:
        addr = None

    json_map = results_business_page.find('div', class_ = "lightbox-map hidden").get('data-map-state')
    dict_map = json.loads(json_map)
    latitude = dict_map["markers"][1]["location"]["latitude"]
    longitude = dict_map["markers"][1]["location"]["longitude"]

    category= results_business_page.find("span", class_ = 'category-str-list').find('a').get_text()
        
    try:
        price_range = results_business_page.find("span", class_ = 'business-attribute price-range').get_text()
    except:
        price_range = None
        
    ratings = results_business_page.find("div", itemprop = 'aggregateRating')
    rating_value = ratings.find("meta").get('content')
    review_count = ratings.find("span").get_text()
    
    try:
        phone_number = results_business_page.find('span',itemprop = "telephone").get_text().strip()
    except:
        phone_number = None
        
    try:
        open_ = results_business_page.find('span', class_ = "nowrap extra open").get_text()
    except:
        open_ = None
    try:
        web = results_business_page.find('span', class_ = "biz-website js-biz-website js-add-url-tagging").find('a').get_text()
    except:
        web = None
        
    return(addr,latitude,longitude,category,price_range,rating_value,review_count,phone_number,open_,web)
    

In [54]:
#test the function
restaurant_url = "https://www.yelp.com/biz/cafe-mogador-brooklyn?osq=Restaurants"
get_info(restaurant_url)

('133 Wythe AveBrooklyn, NY 11211',
 40.719747,
 -73.959993,
 'Moroccan',
 '$$',
 '4.5',
 '1004',
 '(718) 486-9222',
 'Open now',
 'cafemogador.com/williamsburg')